In [1]:
import os
import sys
import cv2
from fer import FER
from your_face_detector_module import FaceDetector  # Replace with your actual module for FaceDetector
from your_video_module import VideoCamera  # Replace with your actual video camera class/module

def check_choice():
    """Prompt user for input and validate the choice."""
    while True:
        try:
            choice = int(input("Enter your choice: "))
            if choice in [1, 2, 3]:
                return choice
            else:
                print("Invalid choice. Please select 1, 2, or 3.")
        except ValueError:
            print("Invalid input. Please enter an integer.")

def analyze_emotions_with_fer(face_img):
    """Analyze emotions from a face image using FER."""
    try:
        emotion_detector = FER(mtcnn=True)  # Using MTCNN for better face detection
        analysis = emotion_detector.detect_emotions(face_img)
        if analysis and len(analysis) > 0:
            emotion, score = max(analysis[0]['emotions'].items(), key=lambda x: x[1])
            return emotion, score
        return "Unknown", 0
    except Exception as e:
        print(f"Emotion analysis failed: {e}")
        return "Unknown", 0

def recognize_people_with_emotions(people_folder, shape):
    """Start recognizing people and analyzing emotions in a live stream."""
    try:
        people = [person for person in os.listdir(people_folder)]
    except:
        print("Have you added at least one person to the system?")
        sys.exit()

    print("These are the people in the Recognition System:")
    for person in people:
        print(f"- {person}")

    detector = FaceDetector('face_recognition_system/frontal_face.xml')
    video = VideoCamera()

    while True:
        frame = video.get_frame()

        # Check if the frame is valid
        if frame is None or frame.size == 0:
            print("Error: Empty frame received. Please check the camera.")
            continue

        faces_coord = detector.detect(frame)
        if len(faces_coord):
            frame, faces_img = get_images(frame, faces_coord, shape)
            for i, face_img in enumerate(faces_img):
                # Predict emotion
                emotion, score = analyze_emotions_with_fer(face_img)

                # Display emotion and confidence score on the video frame
                cv2.putText(frame, f"{emotion.capitalize()} ({score:.2f})",
                            (faces_coord[i][0], faces_coord[i][1] - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        cv2.putText(frame, "Press ESC to exit", (5, frame.shape[0] - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.imshow('Video', frame)

        # Check for ESC key press to exit
        if cv2.waitKey(1) & 0xFF == 27:
            print("Exiting the recognition system...")
            break

    video.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    print(30 * '-')
    print("   POSSIBLE ACTIONS")
    print(30 * '-')
    print("1. Add person to the recognizer system")
    print("2. Start recognizer with emotions")
    print("3. Exit")
    print(30 * '-')

    CHOICE = check_choice()  # Now it's defined

    PEOPLE_FOLDER = "face_recognition_system/people/"
    SHAPE = "rectangle"

    if CHOICE == 1:
        if not os.path.exists(PEOPLE_FOLDER):
            os.makedirs(PEOPLE_FOLDER)
        add_person(PEOPLE_FOLDER, SHAPE)  # Ensure this function is defined elsewhere
    elif CHOICE == 2:
        os.system("sudo modprobe bcm2835-v4l2")  # Required for Raspi-Camera Module (on Raspberry Pi)
        recognize_people_with_emotions(PEOPLE_FOLDER, SHAPE)
    elif CHOICE == 3:
        sys.exit()


ModuleNotFoundError: No module named 'your_face_detector_module'